# Example 3 - Optimisation routine

In Examples 2a and 2b we computed the Curie depth for a single point using a fixed window. Here, we use `CurieOptimise` object to compute the Curie depth over an entire magnetic anomaly.

`CurieOptimise` inherits all methods from `CurieGrid` in addition to:

- iteratively evaluate the Curie depth across the magnetic anomaly
- add priors to limit spurious Curie depth determinations
- optimise Curie depth parameters using constraints from an objective function

### Contents

- [The inverse problem](#The-inverse-problem)
- [Iterate across magnetic anomaly](#Iterate-across-magnetic-anomaly)
- [Compare Bouligand to Tanaka](#Compare-Bouligand-to-Tanaka)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

In [ ]:
# initialise CurieOptimise object
grid = pycurious.CurieOptimise(d, xmin, xmax, ymin, ymax)

window_size = 200e3
xc_list, yc_list = grid.create_centroid_list(window_size)

## The inverse problem

It is notoriously difficult to estimate uncertainties from Curie depth determinations. Intuitively, the fewer points in the power spectrum should result in higher uncertainty, but it is difficult to quantify these uncertainties in practise. For [Bouligand *et al.*, 2009](http://doi.wiley.com/10.1029/2009JB006494) it is difficult to determine the values of $\beta$ and $\Delta z$ since both control the slope of the power spectrum at low wavenumbers. Similarly, for [Tanaka *et al.*, 1999](http://linkinghub.elsevier.com/retrieve/pii/S0040195199000724) the lower and upper ranges of the power spectrum used to compute $z_b$ and $z_t$ is highly subjective and can result in significantly different Curie depths.

Here, we aim to assess the uncertainty of Curie depth determinations using a sensitivity analysis. The approach we outline here is not necessarily the most statistically robust methodology, but it is intended as a practical way forward to compute Curie depth while considering the high degree of uncertainty.